In [79]:
import pandas as pd
import csv
import numpy as np
import os

In [80]:
from tqdm import tqdm
tqdm.pandas(desc="Progress:")

/Users/gagekrumbach/.local/share/virtualenvs/Solar_Forcaster-m3btlaz4/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Create the lambda function that cleans up floating point data.

`29.37s` => `29.37`

In [81]:
import glob

data_path = 'LCD_Data/WI/data/*.csv'
clean_float = lambda x: ''.join(filter(lambda y: y.isdigit() or y == '.', x))

raw_data = pd.concat([pd.read_csv(file_path,
                            usecols=['STATION', 'DATE', 'HourlySkyConditions', 'HourlyStationPressure',
                                'HourlyVisibility', 'HourlyPresentWeatherType', 'HourlyPrecipitation'],
                            parse_dates=['DATE'],
                            dtype={'HourlySkyConditions': str, 'HourlyPresentWeatherType': str},
                            converters={'HourlyStationPressure': clean_float, 'HourlyVisibility': clean_float,
                                        'HourlyPrecipitation': clean_float})
                         for file_path in glob.iglob(data_path)])
raw_data.shape

/Users/gagekrumbach/.local/share/virtualenvs/Solar_Forcaster-m3btlaz4/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(1397040, 7)

Sample data for faster development

In [82]:
w_data = raw_data
#w_data = w_data.sample(n=10000)
w_data.head()

,STATION,DATE,HourlyPrecipitation,HourlyPresentWeatherType,HourlySkyConditions,HourlyStationPressure,HourlyVisibility
0,72233254953,2017-01-01 00:15:00,,NaN,OVC:08 110,28.18,10.00
1,72233254953,2017-01-01 00:35:00,,NaN,BKN:07 60 BKN:07 65 OVC:08 100,28.18,10.00
2,72233254953,2017-01-01 00:55:00,,NaN,BKN:07 55 BKN:07 70 OVC:08 100,28.18,10.00
3,72233254953,2017-01-01 01:15:00,,NaN,SCT:04 55 OVC:08 90,28.17,10.00
4,72233254953,2017-01-01 01:35:00,,NaN,SCT:04 50 OVC:08 90,28.18,10.00


Convert station ID to shorter form

In [83]:
w_data['STATION'] = w_data['STATION'].map(lambda x: int(str(x)[-5:]))
w_data['STATION'].head()

0    54953
1    54953
2    54953
3    54953
4    54953
Name: STATION, dtype: int64

Convert weather codes to a more less specific form that can be used by the forcast api later. Codes are in the format `AU | AW | MW`. They need to be catagroized into `freezing_rain_heavy, freezing_rain, freezing_rain_light, freezing_drizzle, ice_pellets_heavy, ice_pellets, ice_pellets_light, snow_heavy, snow, snow_light, flurries, tstorm, rain_heavy, rain, rain_light, drizzle, fog_light, fog, cloudy, mostly_cloudy, partly_cloudy, mostly_clear, clear`.

In [84]:
import json
with open('weather_lookup_converter.json') as json_file:
    lookup = json.load(json_file)

def weather_decoder(value):
    if pd.notnull(value):
        for weather_type in lookup:
            for code in lookup[weather_type]:
                if code in value:
                    return weather_type

w_data['weather_type'] = w_data.progress_apply(lambda row: weather_decoder(row.HourlyPresentWeatherType), axis=1)

Progress:: 100%|██████████| 1397040/1397040 [00:33<00:00, 41107.00it/s]


Breaking up hourly sky conditions column to a cloud string, cloud base height, and cloud cover using another lookup table

In [85]:
with open('cloud_lookup.json') as json_file:
    lookup = json.load(json_file)

def cloud_decoder(value):
    if pd.notnull(value):
        for cloud_type in lookup:
            if cloud_type in value:
                return lookup[cloud_type]
    return lookup["CLR"]
            
w_data['cloud_type'] = w_data.progress_apply(lambda row: cloud_decoder(row.HourlySkyConditions)["cloud_str"], axis=1)
w_data['cloud_cover'] = w_data.progress_apply(lambda row: cloud_decoder(row.HourlySkyConditions)["cloud_cover"], axis=1)

Progress:: 100%|██████████| 1397040/1397040 [00:34<00:00, 40215.93it/s]


Combine station data (lat, long, elevation) with historic weather data.

In [86]:
station_meta_lookup = pd.read_csv('LCD_Data/WI/stations.csv',
                                usecols=["STATION_ID", "STATION", "LATITUDE", "LONGITUDE", "ELEVATION_(M)"],
                               dtype={"LATITUDE": float, "LONGITUDE": float})

def get_matching_row(value):
    for i, row in enumerate(station_meta_lookup["STATION_ID"]):
        if int(row[5:]) == value:
            return i

w_data["latitude"] = w_data.progress_apply(lambda row:
                                      station_meta_lookup["LATITUDE"][get_matching_row(row.STATION)], axis=1)
w_data["longitude"] = w_data.progress_apply(lambda row:
                                      station_meta_lookup["LONGITUDE"][get_matching_row(row.STATION)], axis=1)
w_data["elevation"] = w_data.progress_apply(lambda row:
                                      station_meta_lookup["ELEVATION_(M)"][get_matching_row(row.STATION)], axis=1)
w_data.head()

Progress:: 100%|██████████| 1397040/1397040 [01:26<00:00, 16175.02it/s]


,STATION,DATE,HourlyPrecipitation,HourlyPresentWeatherType,HourlySkyConditions,HourlyStationPressure,HourlyVisibility,weather_type,cloud_type,cloud_cover,latitude,longitude,elevation
0,54953,2017-01-01 00:15:00,,NaN,OVC:08 110,28.18,10.00,None,cloudy,1.0,45.46917,-89.80556,452.9
1,54953,2017-01-01 00:35:00,,NaN,BKN:07 60 BKN:07 65 OVC:08 100,28.18,10.00,None,cloudy,1.0,45.46917,-89.80556,452.9
2,54953,2017-01-01 00:55:00,,NaN,BKN:07 55 BKN:07 70 OVC:08 100,28.18,10.00,None,cloudy,1.0,45.46917,-89.80556,452.9
3,54953,2017-01-01 01:15:00,,NaN,SCT:04 55 OVC:08 90,28.17,10.00,None,cloudy,1.0,45.46917,-89.80556,452.9
4,54953,2017-01-01 01:35:00,,NaN,SCT:04 50 OVC:08 90,28.18,10.00,None,cloudy,1.0,45.46917,-89.80556,452.9


Use altair to visulize where the statations are. Each stattion has an average of about 10 mi for range. We will need to use spatial interpolation to fill the data.

In [87]:
import altair as alt

state = alt.topo_feature('LCD_Data/WI/WI.geo.json', 'collection')

# US states background
background = alt.Chart(state).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa')

# airport positions on background
points = alt.Chart(station_meta_lookup).mark_circle(
    size=50,
    color='red'
).encode(
    longitude='LONGITUDE:Q',
    latitude='LATITUDE:Q',
    tooltip=['STATION:N'])

background + points

alt.LayerChart(...)

Load the NREL api to get solar data for each point.

In [88]:
def create_url(lat, lon, year, api_key, attributes, your_name, reason_for_use, your_affiliation, your_email):
    return 'http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day=false&interval=60&utc=false&full_name={name}&email={email}&affiliation={affiliation}&mailing_list=false&reason={reason}&api_key={api}&attributes={attributes}'\
    .format(year=year, lat=lat, lon=lon,
        name=your_name, email=your_email,
        affiliation=your_affiliation,
        reason=reason_for_use, api=api_key,
        attributes=attributes)

import time  
data_year = '2017'
for step, point in enumerate(tqdm(station_meta_lookup.iterrows())):    
    url = create_url(lat=str(point[1][2]),
                     lon=str(point[1][3]),
                     api_key=os.getenv('MY_VAR'),
                     attributes="air_temperature,dew_point,dhi,dni,ghi,relative_humidity,wind_direction,wind_speed",
                     year=data_year,
                     your_name='Gage+Krumbach',
                     reason_for_use='demo',
                     your_affiliation='my+institution',
                     your_email='gkrumbac@redhat.com')
    output = pd.read_csv(url, skiprows=2)
    output["STATION"] = int(point[1][0][5:])
    output["DATE"] =pd.date_range('1/1/{yr}'.format(yr=data_year),
                                                    freq='60'+'Min',
                                                    periods=525600/60)
    
    if step==0:
        solar_data = output
    else:
        solar_data = solar_data.append(output)
    
    time.sleep(3)

64it [07:55,  7.44s/it]


In [89]:
df_merge_asof = pd.merge_asof(solar_data.sort_values(by=['DATE']), w_data.sort_values(by=['DATE']),
              on='DATE',
              by='STATION',
             direction='nearest')

In [90]:
is_station = df_merge_asof['STATION']==94929
df_merge_asof[is_station].loc[(df_merge_asof[is_station]['DATE'] > '01-01-2010') & (df_merge_asof[is_station]['DATE'] < '01-2-2010')]

,Year,Month,Day,Hour,Minute,Temperature,Dew Point,DHI,DNI,GHI,...,HourlyPresentWeatherType,HourlySkyConditions,HourlyStationPressure,HourlyVisibility,weather_type,cloud_type,cloud_cover,latitude,longitude,elevation


In [91]:
df_merge_asof = df_merge_asof[['STATION', 'DATE', 'latitude', 'longitude', 'elevation',
                               'Temperature', 'Dew Point', 'Relative Humidity', 'HourlyStationPressure',
                               'Wind Direction', 'Wind Speed', 'HourlyVisibility', 'weather_type',
                               'cloud_type', 'cloud_cover', 'DHI', 'DNI', 'GHI'
                              ]]
df_merge_asof.head()

,STATION,DATE,latitude,longitude,elevation,Temperature,Dew Point,Relative Humidity,HourlyStationPressure,Wind Direction,Wind Speed,HourlyVisibility,weather_type,cloud_type,cloud_cover,DHI,DNI,GHI
0,4864,2017-01-01,45.15417,-89.11056,463.9,-12,-13,79.65,28.17,232.1,3.9,10.00,None,cloudy,1.0,0,0,0
1,4898,2017-01-01,43.42639,-88.70306,285.3,-12,-13,90.69,28.93,234.9,2.3,10.00,None,clear,0.0,0,0,0
2,94940,2017-01-01,43.96667,-90.73333,252.7,-11,-14,76.42,29.00,219.6,4.2,10.00,None,clear,0.0,0,0,0
3,94854,2017-01-01,42.61667,-89.03333,246.3,-9,-10,98.56,29.09,251.0,1.5,10.00,None,clear,0.0,0,0,0
4,4895,2017-01-01,44.55000,-89.53333,338.3,-12,-14,79.90,28.67,234.6,3.3,10.00,None,cloudy,1.0,0,0,0


In [92]:
df_merge_asof.to_csv("out.csv",index=False)
